In [36]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pygame
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import DepthwiseConv2D

# Custom function to handle unsupported 'groups' argument in DepthwiseConv2D
def custom_depthwise_conv2d(*args, **kwargs):
    kwargs.pop("groups", None)  # Remove 'groups' if present
    return DepthwiseConv2D(*args, **kwargs)

# Load the model with custom handling for DepthwiseConv2D
custom_objects = {"DepthwiseConv2D": custom_depthwise_conv2d}
model = load_model(r"C:\Users\egymo\Desktop\ML_Project\Project\best_model.h5", custom_objects=custom_objects, compile=False)

# Save the modified model (optional)
model.save('modified_model.h5')

class EmotionDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Emotion Detection App")
        self.root.geometry("900x700")

        # Center the application window on the screen
        window_width = 650
        window_height = 600
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        position_top = int(screen_height / 2 - window_height / 2)
        position_right = int(screen_width / 2 - window_width / 2)
        self.root.geometry(f"{window_width}x{window_height}+{position_right}+{position_top}")

        # Initialize pygame for music
        pygame.mixer.init()

        # Load your emotion detection model
        self.model = model  # Use the modified model

        # Define emotion-to-music mapping
        self.emotion_to_music = {
            "Angry": r"C:\Users\egymo\Desktop\ML_Project\Project\angry_song.mp3",
            "Disguist": r"C:\Users\egymo\Desktop\ML_Project\Project\Disguist  Disguist_song.mp3",
            "Fear": r"C:\Users\egymo\Desktop\ML_Project\Project\Fear_song.mp3.mp3",
            "Happy": r"C:\Users\egymo\Desktop\ML_Project\Project\pharrell_williams_-_happy_(official_music_video).mp3",
            "Neutral": r"C:\Users\egymo\Desktop\ML_Project\Project\Neutral_song.mp3",
            "Sad": r"C:\Users\egymo\Desktop\ML_Project\Project\Sad_song.mp3",
            "Surprise": r"C:\Users\egymo\Desktop\ML_Project\Project\Surprise_song.mp3",
        }

        # GUI Components
        self.is_paused = False  # State to track whether the music is paused

        # Top frame for title
        self.top_frame = tk.Frame(root, bg="navy", height=100)
        self.top_frame.pack(fill=tk.X)

        self.title_label = tk.Label(self.top_frame, text="Emotion Detection App", font=("Helvetica", 24, "bold"), fg="white", bg="navy")
        self.title_label.pack(pady=20)

        # Main frame for content
        self.main_frame = tk.Frame(root, bg="lightgray")
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

        # Photo display
        self.photo_label = tk.Label(self.main_frame, text="Photo will appear here", bg="white", width=40, height=15, relief="solid", borderwidth=2)
        self.photo_label.grid(row=0, column=0, columnspan=3, padx=20, pady=10)

        # Buttons
        self.upload_button = tk.Button(self.main_frame, text="Attach Photo", command=self.attach_photo, bg="dodgerblue", fg="white", font=("Arial", 12, "bold"), width=15)
        self.upload_button.grid(row=1, column=0, padx=20, pady=10)

        self.detect_button = tk.Button(self.main_frame, text="Detect Emotion", command=self.detect_emotion, bg="green", fg="white", font=("Arial", 12, "bold"), width=15)
        self.detect_button.grid(row=1, column=1, padx=20, pady=10)

        self.webcam_button = tk.Button(self.main_frame, text="Open Webcam", command=self.capture_from_webcam, bg="orange", fg="white", font=("Arial", 12, "bold"), width=15)
        self.webcam_button.grid(row=1, column=2, padx=20, pady=10)

        self.pause_button = tk.Button(self.main_frame, text="Pause Music", command=self.pause_music, bg="yellow", font=("Arial", 12), width=15)
        self.pause_button.grid(row=2, column=0, padx=20, pady=10)

        self.resume_button = tk.Button(self.main_frame, text="Resume Music", command=self.resume_music, bg="green", fg="white", font=("Arial", 12, "bold"), width=15)
        self.resume_button.grid(row=2, column=1, padx=20, pady=10)

        self.stop_button = tk.Button(self.main_frame, text="Stop Music", command=self.stop_music, bg="red", fg="white", font=("Arial", 12, "bold"), width=15)
        self.stop_button.grid(row=2, column=2, padx=20, pady=10)

        # Emotion result
        self.result_label = tk.Label(self.main_frame, text="Emotion: None", font=("Helvetica", 18), bg="lightgray")
        self.result_label.grid(row=3, column=0, columnspan=3, pady=20)

        # Status label
        self.status_label = tk.Label(self.main_frame, text="Status: Ready", font=("Helvetica", 14), bg="lightgray")
        self.status_label.grid(row=4, column=0, columnspan=3, pady=10)

        # Configure grid layout
        self.main_frame.grid_columnconfigure(0, weight=1)
        self.main_frame.grid_columnconfigure(1, weight=1)
        self.main_frame.grid_columnconfigure(2, weight=1)

    def attach_photo(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        if self.file_path:
            try:
                image = Image.open(self.file_path)
                image.thumbnail((300, 300))
                photo = ImageTk.PhotoImage(image)
                self.photo_label.configure(image=photo, text="")
                self.photo_label.image = photo
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load image: {e}")

    def capture_from_webcam(self):
        # Open the default camera .
        cam = cv2.VideoCapture(0)

        # Create a window named "Webcam" to display the video feed.
        cv2.namedWindow("Webcam")

        # Initialize a counter to keep track of the number of screenshots taken.
        img_counter = 0

        # infinite loop to continuously capture frames from the webcam.
        while True:
        # Capture a single frame from the webcam. `ret` indicates if the capture was successful, 
        # and `frame` contains the captured image.
            ret, frame = cam.read()

            # If `ret` is False, it means the frame was not successfully captured.
            if not ret:
                print("Failed to grab frame")  
                break  # Exit the loop if frame capture fails.

            # Display the captured frame in the "Webcam" window.
            cv2.imshow("Webcam", frame)

            # stores the ASCII value of the key pressed.
            k = cv2.waitKey(1) & 0xFF == ord('q')

            # Check if the Esc key (ASCII value 27) was pressed.
            if k % 256 == 27:
                print("Esc hit, closing the app")  
                break  

            # Check if the Space key (ASCII value 32) was pressed.
            elif k % 256 == 32:
                # Create a filename for the screenshot using the counter value.
                img_name = "opencv_frame_{}.png".format(img_counter)

                # Save the current frame as an image file with the generated filename.
                cv2.imwrite(img_name, frame)
                print("Screenshot taken")

                # Increment the screenshot counter for the next file name.
                img_counter += 1
                

        cam.release()
        cv2.destroyAllWindows()

    def preprocess_image(self, file_path):
        image = cv2.imread(file_path)
        image = cv2.resize(image, (224, 224))
        image = image / 255.0
        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[-1] == 1:
            image = np.repeat(image, 3, axis=-1)
        image = np.expand_dims(image, axis=0)
        return image

    def detect_emotion(self):
        if hasattr(self, "file_path"):
            image = self.preprocess_image(self.file_path)
            prediction = self.model.predict(image)
            emotion_index = np.argmax(prediction)
            emotion_classes = ["Angry", "Disguist", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

            if emotion_index < len(emotion_classes):
                detected_emotion = emotion_classes[emotion_index]
                self.result_label.config(text=f"Emotion: {detected_emotion}")
                self.play_music(detected_emotion)
            else:
                messagebox.showerror("Error", "Invalid emotion detected. Please check your model and classes.")
        else:
            messagebox.showwarning("Error", "No photo uploaded!")

    def play_music(self, emotion):
        if emotion in self.emotion_to_music:
            music_file = self.emotion_to_music[emotion]
            pygame.mixer.music.load(music_file)
            pygame.mixer.music.play()
        else:
            messagebox.showinfo("Info", "No music available for this emotion.")

    def pause_music(self):
        if pygame.mixer.music.get_busy() and not self.is_paused:
            pygame.mixer.music.pause()
            self.is_paused = True
            self.status_label.config(text="Status: Music Paused")

    def resume_music(self):
        if self.is_paused:
            pygame.mixer.music.unpause()
            self.is_paused = False
            self.status_label.config(text="Status: Music Resumed")
        elif not pygame.mixer.music.get_busy():
            self.status_label.config(text="Status: No music to resume")

    def stop_music(self):
        if pygame.mixer.music.get_busy():
            pygame.mixer.music.stop()
            self.is_paused = False
            self.status_label.config(text="Status: Music Stopped")


if __name__ == "__main__":
    root = tk.Tk()
    app = EmotionDetectionApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 727ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\egymo\AppData\Local\Temp\ipykernel_14224\2985881692.py", line 178, in detect_emotion
    image = self.preprocess_image(self.file_path)
  File "C:\Users\egymo\AppData\Local\Temp\ipykernel_14224\2985881692.py", line 167, in preprocess_image
    image = cv2.resize(image, (224, 224))
cv2.error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\egymo\AppData\Lo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
